In [1]:
# !pip install Telethon==1.24.0

In [2]:
import time

from datetime import (
    datetime, timezone
)

import asyncio
import pandas as pd
from tqdm.notebook import tqdm

from telethon import (
    TelegramClient, functions
)

# 1. Служебные функции

In [3]:
async def get_comment_data(message, comment, channel_name, channel_orientation):
    async with semaphore_comment:
        return {
            "channel_name": channel_name,
            "channel_orientation": channel_orientation,
            "message_id": message.id,
            "comment_id": comment.id,
            "comment_date": comment.date,
            "comment_edit_date": comment.edit_date,
            "comment_message": comment.message,
            "comment_media": comment.media is not None
        }

In [4]:
async def get_channel_message_data(semaphore_message, semaphore_comment, client, channel, message, channel_name, channel_orientation):
    async with semaphore_message:
        message_data = {
            "body": {
                "channel_name": channel_name,
                "channel_orientation": channel_orientation,
                "comments_unavailable": False
            },
            "comments": []
        }

        message_data["body"].update(
            {
                "id": message.id,
                "date": message.date,
                "edit_date": message.edit_date,
                "message": message.message,
                "views": message.views,
                "forwards": message.forwards,
                "media": message.media is not None,
                "legacy": message.legacy
            }
        )

        # if message.legacy:
        try:
            async for comment in client.iter_messages(channel, reply_to=message.id):
                message_data["comments"].append(
                    # limited parse
                    await get_comment_data(
                        message, comment, channel_name, channel_orientation
                    )
                )
        except:
            message_data["body"].update(
                {
                    "comments_unavailable": True
                }
            )

        message_data["body"].update(
            {
                "comments_count": len(message_data["comments"])
            }
        )

        return message_data

    
async def run_coorutines_progress_bar(coorutines):
    tasks = []
    
    for coorutine in coorutines:
        task = asyncio.create_task(coorutine)
        tasks.append(task)

    tasks_tqdm = [
        await task
        for task in tqdm(
            asyncio.as_completed(tasks), 
            total=len(tasks)
        )
    ]

    return tasks_tqdm

# 2. Константы

In [5]:
# insert your data from "https://my.telegram.org/apps"
API_ID = 1488
API_HASH = 'YOUR HASH'

THRESHOLD_DATE = datetime(
    2022, 2, 22, 0, 0, 0, 
    tzinfo=timezone.utc
)

PRO_RUSSIAN_CHANNELS = [
    "readovkanews", 
    "bbbreaking", 
    "breakingmash"
]

PRO_UKRAINIAN_CHANNELS = [
    "nexta_live", 
    "voynareal", 
    "u_now"
]

# 3. Выгружаем данные

In [6]:
client = TelegramClient('topic_modeling_research', API_ID, API_HASH)
await client.start()

## 3.1. Создаем коорутины

In [7]:
# for "(HTTP code 429)" error
semaphore_message = asyncio.Semaphore(10)
semaphore_comment = asyncio.Semaphore(10)

In [8]:
coorutines = []

for channel_to_parse in tqdm(PRO_RUSSIAN_CHANNELS):
    channel = await client.get_entity(channel_to_parse)
    
    async for message in client.iter_messages(channel):
        if message.date < THRESHOLD_DATE:
            break

        message_coor = get_channel_message_data(
            semaphore_message, semaphore_comment,
            client, channel, message, 
            channel_to_parse, "rus"
        )

        coorutines.append(message_coor)
        
print(
    len(coorutines)
)

  0%|          | 0/3 [00:00<?, ?it/s]

5490


In [9]:
for channel_to_parse in tqdm(PRO_UKRAINIAN_CHANNELS):
    channel = await client.get_entity(channel_to_parse)
    
    async for message in client.iter_messages(channel):
        if message.date < THRESHOLD_DATE:
            break

        message_coor = get_channel_message_data(
            semaphore_message, semaphore_comment,
            client, channel, message, 
            channel_to_parse, "ukr"
        )

        coorutines.append(message_coor)
        
print(
    len(coorutines)
)

  0%|          | 0/3 [00:00<?, ?it/s]

23043


## 3.2. Парсим

In [10]:
parsed_data = await run_coorutines_progress_bar(
    coorutines
)

messages = [
    data["body"] for data in parsed_data
]

comments = [
    data["comments"] for data in parsed_data
]

comments = sum(comments, [])

  0%|          | 0/23043 [00:00<?, ?it/s]

Unhandled error while processing msgs
Traceback (most recent call last):
  File "/Users/erqups/anaconda3/lib/python3.7/site-packages/telethon/network/mtprotosender.py", line 540, in _recv_loop
    await self._process_message(message)
  File "/Users/erqups/anaconda3/lib/python3.7/site-packages/telethon/network/mtprotosender.py", line 555, in _process_message
    await handler(message)
  File "/Users/erqups/anaconda3/lib/python3.7/site-packages/telethon/network/mtprotosender.py", line 634, in _handle_container
    await self._process_message(inner_message)
  File "/Users/erqups/anaconda3/lib/python3.7/site-packages/telethon/network/mtprotosender.py", line 555, in _process_message
    await handler(message)
  File "/Users/erqups/anaconda3/lib/python3.7/site-packages/telethon/network/mtprotosender.py", line 601, in _handle_rpc_result
    if not isinstance(reader.tgread_object(), upload.File):
  File "/Users/erqups/anaconda3/lib/python3.7/site-packages/telethon/extensions/binaryreader.py", 

Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid bu

Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid bu

Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid buffer Invalid response buffer (HTTP code 429)
Invalid bu

## 3.3. Выгружаем

In [12]:
messages = pd.DataFrame(messages)

print(messages.shape)
messages.head(5)

(23043, 12)


,channel_name,channel_orientation,comments_unavailable,id,date,edit_date,message,views,forwards,media,legacy,comments_count
0,readovkanews,rus,False,27683,2022-03-08 11:07:02+00:00,2022-03-08 11:07:13+00:00,"❗️В МЧС РФ заявили, что Россия доставила на Ук...",102751.0,2.0,False,True,106
1,readovkanews,rus,False,27685,2022-03-08 11:39:16+00:00,2022-03-08 11:39:37+00:00,❗️Британская нефтегазовая компания British Pet...,88669.0,106.0,False,True,155
2,readovkanews,rus,False,27682,2022-03-08 10:49:32+00:00,2022-03-08 10:49:44+00:00,"СК РФ установит украинских должностных лиц, пр...",108785.0,69.0,False,True,124
3,readovkanews,rus,False,27688,2022-03-08 12:08:45+00:00,2022-03-08 12:08:57+00:00,В Белграде выразили поддержку России — фанаты ...,71555.0,343.0,True,True,185
4,readovkanews,rus,False,27690,2022-03-08 12:19:26+00:00,2022-03-08 12:31:35+00:00,❗️Крупнейшая в мире криптобиржа Binance с 9 ма...,69515.0,820.0,False,True,238


In [13]:
messages["channel_name"].value_counts()

u_now           8258
voynareal       5006
nexta_live      4289
bbbreaking      3148
readovkanews    1407
breakingmash     935
Name: channel_name, dtype: int64

In [14]:
comments = pd.DataFrame(comments)

print(comments.shape)
comments.head(5)

(418018, 8)


,channel_name,channel_orientation,message_id,comment_id,comment_date,comment_edit_date,comment_message,comment_media
0,readovkanews,rus,27683,1684955,2022-03-08 13:12:02+00:00,NaT,https://t.me/gruz_200_rus,False
1,readovkanews,rus,27683,1684468,2022-03-08 12:43:01+00:00,NaT,"Пизда тупая, ее в этом году не будет",False
2,readovkanews,rus,27683,1684450,2022-03-08 12:41:49+00:00,NaT,Да ты там с рождения,False
3,readovkanews,rus,27683,1683809,2022-03-08 12:14:05+00:00,NaT,Давно уже зарождались разного рода антироссийс...,False
4,readovkanews,rus,27683,1683595,2022-03-08 12:05:36+00:00,NaT,"Курс доллара 200 будет только завтра, и то ско...",False


---

In [15]:
pd.set_option('display.max_rows', 100)

In [16]:
pd.DataFrame(
    messages.groupby(
        by=[
            "channel_name",
            messages["date"].apply(
                lambda value: value.date()
            )
        ]
    )["channel_name"].count()
)

channel_name
channel_name date                    
bbbreaking   2022-02-22           255
             2022-02-23           164
             2022-02-24           533
             2022-02-25           294
             2022-02-26           146
             2022-02-27           221
             2022-02-28           212
             2022-03-01           224
             2022-03-02           242
             2022-03-03           192
             2022-03-04           178
             2022-03-05           165
             2022-03-06            91
             2022-03-07           153
             2022-03-08            78
breakingmash 2022-02-22            32
             2022-02-23            28
             2022-02-24           161
             2022-02-25           114
             2022-02-26            64
             2022-02-27            72
             2022-02-28            65
             2022-03-01            66
             2022-03-02            59
             2022-03-03            56
             2022-03-04            51
             2022-03-05            43
             2022-03-06            47
             2022-03-07            58
             2022-03-08            19
nexta_live   2022-02-22            67
             2022-02-23            65
             2022-02-24           592
             2022-02-25           525
             2022-02-26           532
             2022-02-27           349
             2022-02-28           339
             2022-03-01           278
             2022-03-02           292
             2022-03-03           221
             2022-03-04           291
             2022-03-05           215
             2022-03-06           180
             2022-03-07           247
             2022-03-08            96
readovkanews 2022-02-22            56
             2022-02-23            49
             2022-02-24           181
             2022-02-25           137
             2022-02-26           104
             2022-02-27            93
             2022-02-28           103
             2022-03-01            88
             2022-03-02            93
             2022-03-03           112
             2022-03-04            92
             2022-03-05           101
             2022-03-06            73
             2022-03-07            80
             2022-03-08            45
u_now        2022-02-22            77
             2022-02-23           113
             2022-02-24           557
             2022-02-25           592
             2022-02-26           777
             2022-02-27           780
             2022-02-28           681
             2022-03-01           700
             2022-03-02           710
             2022-03-03           717
             2022-03-04           640
             2022-03-05           569
             2022-03-06           545
             2022-03-07           554
             2022-03-08           246
voynareal    2022-02-22            83
             2022-02-23            52
             2022-02-24           464
             2022-02-25           572
             2022-02-26           552
             2022-02-27           540
             2022-02-28           456
             2022-03-01           380
             2022-03-02           312
             2022-03-03           323
             2022-03-04           348
             2022-03-05           303
             2022-03-06           231
             2022-03-07           244
             2022-03-08           146

In [17]:
pd.DataFrame(
    comments.groupby(
        by=[
            "channel_name",
            comments["comment_date"].apply(
                lambda value: value.date()
            )
        ]
    )["channel_name"].count()
)

channel_name
channel_name comment_date              
nexta_live   2022-02-24           13317
             2022-02-25            4085
             2022-02-26              53
             2022-02-27              27
             2022-02-28              24
             2022-03-01               9
             2022-03-02              91
             2022-03-03               5
             2022-03-04               5
             2022-03-05           10120
             2022-03-06           21833
             2022-03-07           49937
             2022-03-08           19197
readovkanews 2022-02-22            7624
             2022-02-23            7273
             2022-02-24           22704
             2022-02-25           26290
             2022-02-26           24280
             2022-02-27           26160
             2022-02-28           25534
             2022-03-01           26365
             2022-03-02           20202
             2022-03-03           23967
             2022-03-04           17464
             2022-03-05           22336
             2022-03-06           18091
             2022-03-07           18578
             2022-03-08           12447

---

In [18]:
messages.to_parquet(
    "../data/processed/messages.parquet"
)

In [19]:
comments.to_parquet(
    "../data/processed/comments.parquet"
)